In [ ]:
# !pip install -U pip
# !pip install -U setuptools wheel
# !pip install autogluon

In [ ]:
import pandas as pd
import random
import os
import numpy as np
from tqdm import tqdm
import math
import os
import torch
import random
import itertools

import math
import warnings
warnings.filterwarnings(action='ignore')

import pandas as pd
import numpy as np
import random
import os
import matplotlib.pyplot as plt
import warnings; warnings.filterwarnings("ignore")
import shutil
import time
from scipy.stats import gmean
from itertools import combinations
from tqdm import tqdm
tqdm.pandas()

In [ ]:
future_covariates = [
    'temp', 'prec', 'wind', 'hum', 'isolation',
    'sunshine', 'use', 'area_1', 'area_2', 'hour', 'month', 'holiday',
    'sin_hour', 'cos_hour', 'sin_date', 'cos_date',
    'sin_weekday', 'cos_weekday', 'summer_sin', 'summer_cos', 'temp_F',
    'temp2', 'THI', 'WC', 'is_rain', 'log_temp', 'wind_power', 'dew_point',
    'solar_per_hour', 'CDH', 'min_temp', 'max_temp', 'min_wind', 'max_wind',
    'min_hum', 'max_hum', 'mean_THI', 'mean_CDH', 'min_log_temp',
    'max_log_temp', 'mean_WC',
    'target_mean_1', 'target_std_1', 'target_min_1', 'target_max_1', 'z_score',
]

train_df = pd.read_csv('./data/train_p_final.csv')
train_df["log_power"] = np.log(train_df["power"])   # 로그 변환
cat_features = ['holiday','use', 'build_num']
for col in cat_features:
    train_df[col] = train_df[col].astype('category')
num_cols = train_df.select_dtypes(include=["number"]).columns
train_df[num_cols] = train_df[num_cols].clip(lower=-1e6, upper=1e6)
train_df.rename(columns={'build_num':'item_id', 'date_time':'timestamp'}, inplace=True)
train_df.drop(columns=['num_date_time', 'timestamp', 'power'], inplace=True)  # 원래 power는 제거

# predictor 학습 시 label을 'log_power'로 지정
from autogluon.tabular import TabularPredictor
predictor = TabularPredictor(label='log_power', eval_metric='smape').fit(
    train_df,
    presets='best_quality',
    dynamic_stacking=False,
    ag_args_fit={'num_gpus': 1}
)

test_df = pd.read_csv('./data/test_p_final.csv')
test_df.drop(columns=['num_date_time'], inplace=True)
for col in cat_features:
    test_df[col] = test_df[col].astype('category')
num_cols = test_df.select_dtypes(include=["number"]).columns
test_df[num_cols] = test_df[num_cols].clip(lower=-1e6, upper=1e6)
test_df.rename(columns={'build_num':'item_id', 'date_time':'timestamp'}, inplace=True)

# log-space 예측
y_pred_log = predictor.predict(test_df)

sub = pd.read_csv('./sample_submission.csv')
sub['answer'] = np.exp(y_pred_log)
sub.to_csv('./submission/ag_tabular_final.csv', index=False)